# RSA tutorial

In [1]:
%autosave 0
import fidelio_functions

Autosave disabled


In [2]:
%load_ext autoreload
%autoreload 2
from fidelio_functions import *

## The problem
Alice wants to send Bob a message. She does not want anyone else to read it.  
She doesn't have a safe way to send Bob a password.  What can she do?

Alice buys a padlock and keeps the only key. She mails the padlock to Bob.  
Bob locks the message in a sturdy box and mails it to Alice.  
If the lock and box are very difficult to break, then nobody but Alice can read the message.

In [3]:
text = 'WHERE IS RPT WHERE IS TASK FORCE THIRTY FOUR RR THE WORLD WONDERS'

# Convert this to digits
digits = text_to_digits(text)
print(digits)

[55, 40, 37, 50, 37, 0, 41, 51, 0, 50, 48, 52, 0, 55, 40, 37, 50, 37, 0, 41, 51, 0, 52, 33, 51, 43, 0, 38, 47, 50, 35, 37, 0, 52, 40, 41, 50, 52, 57, 0, 38, 47, 53, 50, 0, 50, 50, 0, 52, 40, 37, 0, 55, 47, 50, 44, 36, 0, 55, 47, 46, 36, 37, 50, 51]


## Public-key encryption
To save shipping costs, Alice and Bob decide not to use a physical padlock.  
Instead, Alice sends Bob instructions for creating a mathematical puzzle.  
The puzzle is easy to create but hard to solve without knowing a hint.  

In our case, the puzzle is the [RSA problem](https://en.wikipedia.org/wiki/RSA_problem).  
The instructions are Alice's **public key** $k$ and **RSA number** $n$.  
The secret hint is Alice's **private key** $x$.

## Key generation
The RSA number $n$ is generated by multiplying two randomly-chosen primes.  
The public key is another randomly-chosen prime with some special rules.  
Finding the private key is harder. (Scroll down for an explanation.)

Publicly-known methods for cracking RSA start by factoring $n$ into its prime factors $p*q$. If $n$ is large enough, then figuring out $p$ and $q$ is extremely slow - unless you have a reliable quantum computer, in which case you can use [Shor's algorithm](https://en.wikipedia.org/wiki/Shor%27s_algorithm).

By default, Fidelio uses numbers big enough to be interesting but not big enough to be truly secure.

In [4]:
n, public_key, private_key = generate_keys(verbose=True)

Loading prime numbers from Primes.txt
RSA number is 19379 * 98429 = 1907455591
Euler totient is 1907337784
Testing public key 22271
Public key is 22271
Private key is 1810819007


## RSA encryption
Alice converts the message to a sequence of integers. $M_1, M_2, \ldots$    
For each integer $M$, she creates a cipher $C$ by [modular exponentiation](https://en.wikipedia.org/wiki/Modular_exponentiation):

$$
C = M^k \ \% \ n
$$

Python's build-in [`pow()`](https://docs.python.org/3/library/functions.html#pow) function can do modular exponentiation. Undoing this operation requires solving the [discrete logarithm problem](https://en.wikipedia.org/wiki/Discrete_logarithm_records), which is extremely difficult if you don't know the prime factorization of $n$.

It's important that $M < n$ for all $M$'s in the message.  
By default, Fidelio choose primes between 10K and 100K, which ensures $10^8  < n < 10^{10}$.  
Each packet will be at most 99,999,999, so $M < n$ unless you mess with the defaults.

In [5]:
# Pack these digits into 8-digit blocks (and pad the end if necessary)
packets = packetize(digits)
print(packets)

[55403750, 37004151, 504852, 554037, 50370041, 51005233, 51430038, 47503537, 524041, 50525700, 38475350, 505000, 52403700, 55475044, 36005547, 46363750, 51673926]


In [6]:
# Encrypt each packet
cipher = [ pow(x,public_key,n) for x in packets ]
print( cipher )

[1881228713, 572333103, 1434110925, 41338544, 912276836, 639469899, 698740767, 926488208, 535497995, 1186948106, 1808687922, 151244426, 273313465, 1360879835, 1724078880, 1054240284, 833068773]


In [7]:
# This is what the text looks like after it's been encrypted
digits_to_text(unpacketize(cipher))

'2q6w-Y7A*#.B+)9IAuL{6ls&_~ey)ewHl\'|`x4(UV%+v~q&2(do6/,LJ&;A-N%-\\wC18\'xp*V8"ts>&'

## RSA decryption

Bob raises each $C$ to the power $x$ mod $n$:

$$
M = C^x \ \% \ n
$$

It's not obvious to me why this should equal $M$, but it does. (Scroll down for a proof.)

In [8]:
# Decrypt the packets
decipher = [ pow(x,private_key,n) for x in cipher ]
print(decipher)

[55403750, 37004151, 504852, 554037, 50370041, 51005233, 51430038, 47503537, 524041, 50525700, 38475350, 505000, 52403700, 55475044, 36005547, 46363750, 51673926]


In [9]:
# Unpack and convert back to text
digits_to_text(unpacketize(decipher))

'WHERE IS RPT WHERE IS TASK FORCE THIRTY FOUR RR THE WORLD WONDERS'

## Generating the RSA number and public key

Generating $n$ is easy: just choose two primes, multiply them together, and don't tell anyone what the two prime factors are. Generating a good key pair $k,x$ requires a bit more work. Fidelio uses the **Euler totient method**:

Calculate [Euler's totient](https://en.wikipedia.org/wiki/Euler's_totient_function) $\phi(n) = (p-1)(q-1)$.  

Choose a public key $k$ such that:
- $k$ is prime
- $k < \phi(n)$
- $k$ is not a factor of $\phi(n)$.

Find $x$ such that $kx \ \% \ \phi = 1$. This $x$ is the *multiplicative inverse of $k$ (mod $\phi$).*

Fidelio uses a list of primes from [the University of Utah mathematics department](https://www.math.utah.edu/~pa/math/p10000.html).  
(These primes are too small to be used for professional encryption, but they're OK for practice.)

In [10]:
# Let's use tiny primes for this example
primes = load_primes(too_large=50)
print(primes)

Loading prime numbers from Primes.txt
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]


In [11]:
# Load some prime numbers from a file. Choose p and q at random.
n, totient = choose_rsa_number(primes,verbose=True)

RSA number is 7 * 43 = 301
Euler totient is 252


In [12]:
# Choose a public key which meets the criteria
public_key = choose_public_key(primes,totient,verbose=True)

Testing public key 17
Public key is 17


## Finding the private key
Finding the inverse of $k$ mod $\phi(n)$ is the hard part.

NOT FINISHED YET

In [13]:
# Find the private key x such that kx % n = 1
private_key = find_private_key(public_key,totient,verbose=True)

Private key is 89


In [14]:
# Check our math: this should be 1
(public_key * private_key) % totient

1

## How decryption works
Decrypt each $C$ by exponentiating it to the power $x$ mod $n$:

$$
C^x \ \% \ n
= (M^k)^x \ \% \ n
= M^{kx} \ \% \ n
$$

The prime factorization of $n$ is $pq$. The [Chinese remainder theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) says that $M^{kx} \ \% \ n = M$ if and only if

$$
M^{kx} \ \% \ p = M
\qquad \textrm{AND} \qquad
M^{kx} \ \% \ q = M
$$

Let's do the mod $p$ test first. In the unlikely event that $M$ is a multiple of $p$, we know $M \ \% \ p = 0$ and it's easy:

$$
M^{kx} \ \% \ p
= 0^{kx} \ \% \ p
= 0
$$

What if we aren't that lucky? Recall that we chose $k$ and $x$ such that

$$
kx \ \% \ (p-1)(q-1) = 1
$$

so $kx-1 = h(p-1)(q-1)$ for some $h$. Whatever $h$ is, we know

$$
M^{kx}
= M \cdot M^{kx-1}
= M \cdot M^{h(p-1)(q-1)}
$$

Since $p$ is prime and $M$ is not a multiple of $p$, we can quote [Fermat's Little Theorem](https://en.wikipedia.org/wiki/Fermat's_little_theorem):

$$
M^{p-1} \ \% \ p = 1
$$

which means

$$
M \cdot M^{h(p-1)(q-1)} \ \% \ p
= M \cdot 1^{h(q-1)} \ \% \ p
= M \ \% \ p
$$

To prove that $M^{kx} \ \% \ q = M$, repeat the same logic with $p$ and $q$ trading places.  
In conclusion, $C^x \ \% \ n$ really is the original message $M$.
